# Illustration of the doubling-distance code
This notebook aims at illustrating how a doubling distance code like 

![image.png](./images/doubling_distance.png)

can be implemented using the template approach.

## 1. Define the building blocks

The first step to create a template is to define the base blocks (basically, every arrangement of plaquettes enclosed in a green shape in the image above) that will be used to build it.

The `tqec` package already implement some of the most widely used building blocks in the `tqec.templates.fixed` and `tqec.templates.scalable` modules.

**Important note**: In the following code cell, **ALL** plaquette indices are defined as `1`. This is **not** what we would like to generate for a QEC code, that would require to define different plaquettes. This simplification is done to avoid the clutter and complexity of plaquette numbering.

In [10]:
import numpy
from tqec.templates.atomic.rectangle import (
    AlternatingRectangleTemplate,
    RawRectangleTemplate,
)
from tqec.templates.atomic.square import AlternatingSquareTemplate
from tqec.templates.base import TemplateWithIndices
from tqec.templates.scale import Dimension, LinearFunction

dim = Dimension(2, LinearFunction(2))
nsone = Dimension(1, LinearFunction(0, 1))
nstwo = Dimension(2, LinearFunction(0, 2))
left_white_scalable_rectangle = TemplateWithIndices(
    AlternatingRectangleTemplate(nsone, dim), [1, 0]
)  # 2, 9, 19
right_white_scalable_rectangle = TemplateWithIndices(
    AlternatingRectangleTemplate(nsone, dim), [0, 1]
)  # 4, 13, 23
top_white_scalable_rectangle = TemplateWithIndices(
    AlternatingRectangleTemplate(dim, nsone), [1, 0]
)  # 8
top_black_scalable_rectangle = TemplateWithIndices(
    AlternatingRectangleTemplate(dim, nsone), [0, 1]
)  # 1
bottom_black_scalable_rectangle = TemplateWithIndices(
    AlternatingRectangleTemplate(dim, nsone), [1, 0]
)  # 24, 26
scalable_square = TemplateWithIndices(
    AlternatingSquareTemplate(dim), [1, 1]
)  # 3, 10, 20, 22
fixed_square = TemplateWithIndices(AlternatingSquareTemplate(nstwo), [1, 1])  # 16
horizontal_scalable_rectangle = TemplateWithIndices(
    AlternatingRectangleTemplate(dim, nstwo), [1, 1]
)  # 6, 15, 17
vertical_scalable_rectangle = TemplateWithIndices(
    AlternatingRectangleTemplate(nstwo, dim), [1, 1]
)  # 11, 21
left_fixed_square = TemplateWithIndices(RawRectangleTemplate([[0]]), [1])
right_fixed_square = TemplateWithIndices(RawRectangleTemplate([[0]]), [1])
bottom_fixed_square = TemplateWithIndices(RawRectangleTemplate([[0]]), [1])
# Specific cases:
# - the corner numbered 7
corner = TemplateWithIndices(RawRectangleTemplate([[0]]), [1])

The only building block that is not already defined in the `tqec` package is the one numbered $12$. 

This building block is basically a regular `ScalableAlternatingSquare` with its top-right plaquette replaced by another plaquette. Following this observation, this basic block is implemented by subclassing the `ScalableAlternatingSquare` class and replacing the top-right plaquette by the special plaquette required.

In [11]:
# - the square numbered 12 that has a specific top-right plaquette
class SpecificAlternatingSquareTemplate(AlternatingSquareTemplate):
    def instantiate(
        self, x_plaquette: int, z_plaquette: int, special_plaquette: int, *_: int
    ) -> numpy.ndarray:
        arr = super().instantiate(x_plaquette, z_plaquette, *_)
        arr[0, -1] = special_plaquette
        return arr


top_right_specific_square = TemplateWithIndices(
    SpecificAlternatingSquareTemplate(dim), [1, 1, 1]
)

Now that the building blocks are defined, we create a list of templates according to the numbering in the image at the beginning of this notebook. 

Note that this list contains duplicate instances, meaning that changing in-place one instance in the list *might* also change another item of the list. 
This is not an issue because:
- this list will only be used to initialise the `ComposedTemplate` instance and,
- the only operation in `ComposedTemplate` that modifies the underlying templates stored is when calling `ComposedTemplate.scale_to`, and the same scale is forwarded to all the stored templates.

Nevertheless, you might want to store copies to avoid any subtle reference issues if the `ComposedTemplate` implementation changes in the future. 

**Important note**: lists being 0-indexed in Python, the template indices in Python code will be shifted. As such, the template numbered `1` in the picture above will be the template indexed as `0` in the code below.

In [12]:
templates = [
    # 0
    top_black_scalable_rectangle,
    left_white_scalable_rectangle,
    scalable_square,
    right_white_scalable_rectangle,
    left_fixed_square,
    # 5
    horizontal_scalable_rectangle,
    corner,
    top_white_scalable_rectangle,
    left_white_scalable_rectangle,
    scalable_square,
    # 10
    vertical_scalable_rectangle,
    top_right_specific_square,
    right_white_scalable_rectangle,
    left_fixed_square,
    horizontal_scalable_rectangle,
    # 15
    fixed_square,
    horizontal_scalable_rectangle,
    right_fixed_square,
    left_white_scalable_rectangle,
    scalable_square,
    # 20
    vertical_scalable_rectangle,
    scalable_square,
    right_white_scalable_rectangle,
    bottom_black_scalable_rectangle,
    bottom_fixed_square,
    # 25
    bottom_black_scalable_rectangle,
]
assert len(templates) == 26

## 2. Define the relations

Now that the basic building blocks have been defined, we should encode the relationships between each block to connect them. There are two ways of specifying relationships.

### 2.1. Relative positioning

Building blocks can be positioned relatively to each other. The following code cell defines such relations in a list that contains 3-dimensional tuples containing:
1. The template that should be positioned.
2. The relative position of the template provided in 1.
3. The template used as an anchor to position the template provided in 1.

This order in the tuple has been picked to simplify the input and reading of relationships. 

As such, the first entry of the `relations` list below can be read as "the template numbered `0` is positioned `ABOVE_OF` the template numbered `2`".

In [13]:
from tqec.enums import ABOVE_OF, BELOW_OF, LEFT_OF, RIGHT_OF, CornerPositionEnum

relations = [
    (0, ABOVE_OF, 2),
    (1, LEFT_OF, 2),
    (3, RIGHT_OF, 2),
    (4, BELOW_OF, 1),
    (5, BELOW_OF, 2),
    (9, BELOW_OF, 5),
    (8, LEFT_OF, 9),
    (10, RIGHT_OF, 9),
    (11, RIGHT_OF, 10),
    (12, RIGHT_OF, 11),
    (7, ABOVE_OF, 11),
    (13, BELOW_OF, 8),
    (14, BELOW_OF, 9),
    (15, BELOW_OF, 10),
    (16, BELOW_OF, 11),
    (19, BELOW_OF, 14),
    (18, LEFT_OF, 19),
    (20, RIGHT_OF, 19),
    (21, RIGHT_OF, 20),
    (22, RIGHT_OF, 21),
    (17, ABOVE_OF, 22),
    (23, BELOW_OF, 19),
    (24, RIGHT_OF, 23),
    (25, BELOW_OF, 21),
]

### 2.2. Corner pinning

Relative positioning becomes ambiguous when the dimensions of the two "glued" sides do not match. Most of the template share at least one dimension with their neighbours (for example, the template numbered 5 in the image does not share a dimension with 6, but it does share one with 2), but not all of them (look at template numbered 7 in the picture).

To circumvent that, a second way of specifying relations has been added: corner pinning. This method consists in defining a corner on each of the two templates part of the defined relation, and assume that these two corners are stuck together.

*Note*: the `ComposedTemplate` internally uses the corner pinning method and translates the relations using "relative positioning" to corner pinning. Relative positioning is kept in the interface as another, potentially simpler to read and understand, way of encoding relationships between templates.

In [14]:
corner_relations = [
    ((6, CornerPositionEnum.LOWER_LEFT), (CornerPositionEnum.UPPER_RIGHT, 9)),
]

## 3. Construct the template orchestrator

Now that templates and their relative relationships have been defined, creating the `ComposedTemplate` instance is trivial.

In [15]:
from tqec.templates.composed import ComposedTemplate


doubling_template = ComposedTemplate(templates)
for start, reldir, end in relations:
    doubling_template.add_relation(start, reldir, end)

for (start, start_corner), (end_corner, end) in corner_relations:
    doubling_template.add_corner_relation((start, start_corner), (end, end_corner))

## 4. Have fun

The `ComposedTemplate` instance is now created, you can scale it, display it, just have fun playing with it.

In [16]:
from tqec.display import display_template

display_template(doubling_template)

  .  .  1  .  1  .  .  .  .  .  .  .
  1  1  1  1  1  .  .  .  .  .  .  .
  .  1  1  1  1  1  .  .  .  .  .  .
  1  1  1  1  1  .  .  .  .  .  .  .
  .  1  1  1  1  1  .  .  .  .  .  .
  1  1  1  1  1  .  .  .  .  .  .  .
  .  1  1  1  1  1  .  1  .  1  .  .
  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1
  .  1  .  1  .  1  .  1  .  1  .  .


In [17]:
display_template(doubling_template.scale_to(3))

  .  .  1  .  1  .  1  .  .  .  .  .  .  .  .  .
  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .  .
  .  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .
  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .  .
  .  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .
  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .  .
  .  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .
  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .  .
  .  1  1  1  1  1  1  1  .  1  .  1  .  1  .  .
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  

In [18]:
plaquette_indices = list(range(1, doubling_template.expected_plaquettes_number + 1))
arr = doubling_template.scale_to(500).instantiate(*plaquette_indices)
print("Array shape:", arr.shape)

Array shape: (3006, 2004)
